<a href="https://colab.research.google.com/github/SulemanDawoodAhmad/ML-Crypto-Currency-Prediction-Project/blob/main/Crypto_Currency_SL_TP_Prediction_Project_Data_Processing_%26_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from platform import python_version 
python_version()

'3.7.13'

In [ ]:
%%capture

#Installing the Python Binance API and talib Libraries

!pip install python-binance

!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
!pip install swifter
!pip instal tensorflow

In [ ]:
#Import Libraries

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime, timezone, timedelta
from dateutil import tz
from dateutil.relativedelta import relativedelta
import tqdm as tqdm
from binance.client import Client
import talib as ta
import swifter as sft

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd '/content/gdrive/My Drive/Colab Notebooks'

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks


In [ ]:
#Setting up Keys for Binance API Data Calls
bin_api_key = '#hidden due to private nature of key' 
bin_api_secret = 'hidden due to private nature of key'
client = Client(bin_api_key, bin_api_secret)

In [ ]:
#Importing the Crypto Currency Data and Storing in a Data Frame

US = tz.gettz('America/Chicago')
months = 12
start = str((datetime.today().astimezone(US) - relativedelta(months=months)).strftime('%b %d, %Y'))
end = str(datetime.today().astimezone(US).strftime('%b %d, %Y'))

pairs = ['BTCUSDT','ETHUSDT','BNBUSDT','ADAUSDT','SOLUSDT','LUNAUSDT','DOTUSDT','MATICUSDT','ATOMUSDT','TRXUSDT','XRPUSDT','AVAXUSDT','DOGEUSDT','SHIBUSDT','LINKUSDT']

price_data=pd.DataFrame()

for p in pairs:
    history_request=client.get_historical_klines(symbol=p,interval=client.KLINE_INTERVAL_15MINUTE,start_str=start,end_str=end) 
    col_names = ['OpenTime','open', 'high', 'low', 'close', 'volume','CloseTime', 'qav','num_trades','taker_base_vol','taker_quote_vol', 'ignore']
    history_data=pd.DataFrame(history_request, columns=col_names)
    history_data['pair']=p
    price_data=price_data.append(history_data)

    time.sleep(10)

price_data['OpenTime'] = [datetime.fromtimestamp(ts / 1000) for ts in price_data['OpenTime']]
price_data['CloseTime'] = [datetime.fromtimestamp(ts / 1000) for ts in price_data['CloseTime']]

price_data=price_data.loc[:,['OpenTime','open','high','low','close','volume','pair']]
price_data['date_time']=pd.to_datetime(price_data['OpenTime'])
price_data=price_data.drop('OpenTime', axis=1)
price_data['date'] = [d.date() for d in price_data["date_time"]]
price_data['date'] = pd.to_datetime(price_data['date'], format='%Y-%m-%d')
price_data['time'] = [d.time() for d in price_data["date_time"]]
columns=['date_time','date','time','open','high','low','close', 'volume', 'pair']
price_data=price_data.reindex(columns=columns)
price_data=price_data.reset_index(drop=True)

for col in price_data.columns:
      if (not 'date_time' in col) and (not 'date' in col) and (not 'time' in col) and (not 'pair' in col):
          price_data[col] = price_data[col].astype(float)

display(price_data)

KeyboardInterrupt: ignored

In [ ]:
#Sorting the Dataframe by Time for Each Ticker and Generating Training and Test Samples

price_data=price_data.sort_values(['pair','date_time'])
price_data=price_data.reset_index(drop=True)
display(price_data)

,date_time,date,time,open,high,low,close,volume,pair
0,2021-04-08 00:00:00,2021-04-08,00:00:00,1.17356,1.18247,1.16999,1.18247,2858686.2,ADAUSDT
1,2021-04-08 00:15:00,2021-04-08,00:15:00,1.18247,1.19413,1.17972,1.18306,3479725.9,ADAUSDT
2,2021-04-08 00:30:00,2021-04-08,00:30:00,1.18344,1.19108,1.18123,1.18929,1808114.1,ADAUSDT
3,2021-04-08 00:45:00,2021-04-08,00:45:00,1.18933,1.19048,1.18292,1.18681,2074314.9,ADAUSDT
4,2021-04-08 01:00:00,2021-04-08,01:00:00,1.18650,1.18912,1.18120,1.18646,1591845.0,ADAUSDT
...,...,...,...,...,...,...,...,...,...
521712,2022-04-07 23:00:00,2022-04-07,23:00:00,0.78760,0.78880,0.78670,0.78880,1188196.0,XRPUSDT
521713,2022-04-07 23:15:00,2022-04-07,23:15:00,0.78880,0.79000,0.78680,0.78760,1409305.0,XRPUSDT
521714,2022-04-07 23:30:00,2022-04-07,23:30:00,0.78760,0.78920,0.78730,0.78840,884877.0,XRPUSDT
521715,2022-04-07 23:45:00,2022-04-07,23:45:00,0.78840,0.79110,0.78660,0.78710,1646622.0,XRPUSDT


In [ ]:
#Importing Price Data from the Saved Excel File

price_data=pd.read_excel("price_data.xlsx")
price_data.head()

,date_time,date,time,open,high,low,close,volume,pair
0,2021-04-04 00:00:00,2021-04-04,00:00:00,57051.95,57051.95,56507.91,56843.92,1664.516720,BTCUSDT
1,2021-04-04 00:15:00,2021-04-04,00:15:00,56843.92,57131.68,56700.00,57113.85,610.821212,BTCUSDT
2,2021-04-04 00:30:00,2021-04-04,00:30:00,57113.85,57174.21,56875.19,57173.10,411.728153,BTCUSDT
3,2021-04-04 00:45:00,2021-04-04,00:45:00,57174.27,57269.92,56598.07,56600.00,964.425651,BTCUSDT
4,2021-04-04 01:00:00,2021-04-04,01:00:00,56600.00,56830.72,56388.00,56830.72,1694.723501,BTCUSDT


In [ ]:
#Training and Test Split

#Train Split from April 8th, 2021 to December 8th, 2021

# train_data=pd.DataFrame()

# for p in price_data['pair'].unique():
#     price_data_ticker=price_data[price_data['pair']==p]
#     data_chunk=price_data_ticker.loc[(price_data_ticker['date'] >= '2021-04-08') & (price_data_ticker['date'] <= '2021-12-08')]
#     train_data=train_data.append(data_chunk)
#     train_data=train_data.reset_index(drop=True)

#Test Split 1 from December 9th, 2021 to February 8th, 2022

test_data1=pd.DataFrame()

for p in price_data['pair'].unique():
    price_data_ticker=price_data[price_data['pair']==p]
    data_chunk=price_data_ticker.loc[(price_data_ticker['date'] >= '2021-12-09') & (price_data_ticker['date'] <= '2022-02-08')]
    test_data1=test_data1.append(data_chunk)
    test_data1=test_data1.reset_index(drop=True)

#Test Split 2 from February 9th, 2021 to April 8th, 2022

test_data2=pd.DataFrame()

for p in price_data['pair'].unique():
    price_data_ticker=price_data[price_data['pair']==p]
    data_chunk=price_data_ticker.loc[(price_data_ticker['date'] >= '2022-02-09') & (price_data_ticker['date'] <= '2022-04-08')]
    test_data2=test_data2.append(data_chunk)
    test_data2=test_data2.reset_index(drop=True)

# display(train_data)
display(test_data1)
display(test_data2)

,date_time,date,time,open,high,low,close,volume,pair
0,2021-12-09 00:00:00,2021-12-09,00:00:00,50471.19,50797.76,50435.58,50792.67,446.66366,BTCUSDT
1,2021-12-09 00:15:00,2021-12-09,00:15:00,50792.67,50792.68,50323.83,50468.84,337.65212,BTCUSDT
2,2021-12-09 00:30:00,2021-12-09,00:30:00,50468.83,50612.48,50399.22,50570.83,203.23244,BTCUSDT
3,2021-12-09 00:45:00,2021-12-09,00:45:00,50570.83,50670.56,50388.02,50399.83,177.59336,BTCUSDT
4,2021-12-09 01:00:00,2021-12-09,01:00:00,50399.84,50483.68,50265.22,50282.17,241.82873,BTCUSDT
...,...,...,...,...,...,...,...,...,...
89275,2022-02-08 22:45:00,2022-02-08,22:45:00,18.24,18.36,18.23,18.36,21344.38000,LINKUSDT
89276,2022-02-08 23:00:00,2022-02-08,23:00:00,18.36,18.45,18.35,18.45,28615.87000,LINKUSDT
89277,2022-02-08 23:15:00,2022-02-08,23:15:00,18.45,18.54,18.44,18.54,44979.72000,LINKUSDT
89278,2022-02-08 23:30:00,2022-02-08,23:30:00,18.53,18.55,18.44,18.50,28859.29000,LINKUSDT


,date_time,date,time,open,high,low,close,volume,pair
0,2022-02-09 00:00:00,2022-02-09,00:00:00,44043.00,44146.20,44000.00,44115.17,394.10543,BTCUSDT
1,2022-02-09 00:15:00,2022-02-09,00:15:00,44115.17,44193.42,43971.67,44016.59,336.07521,BTCUSDT
2,2022-02-09 00:30:00,2022-02-09,00:30:00,44016.58,44266.76,43958.70,44257.75,501.64743,BTCUSDT
3,2022-02-09 00:45:00,2022-02-09,00:45:00,44257.74,44341.88,44162.02,44243.99,303.39698,BTCUSDT
4,2022-02-09 01:00:00,2022-02-09,01:00:00,44243.99,44247.54,44087.55,44115.31,347.37589,BTCUSDT
...,...,...,...,...,...,...,...,...,...
77770,2022-04-03 23:00:00,2022-04-03,23:00:00,18.16,18.18,18.01,18.13,70741.59000,LINKUSDT
77771,2022-04-03 23:15:00,2022-04-03,23:15:00,18.13,18.19,18.06,18.17,45828.26000,LINKUSDT
77772,2022-04-03 23:30:00,2022-04-03,23:30:00,18.16,18.21,18.05,18.07,49128.80000,LINKUSDT
77773,2022-04-03 23:45:00,2022-04-03,23:45:00,18.07,18.10,18.02,18.08,29731.85000,LINKUSDT


In [ ]:
#Defining Functions to Calculate Summary Statistics and Distribution of Returns

def summary_stats(x, data, columns, column, window):
    matrix = np.array(data.iloc[x.name+1:x.name+1+window][columns])
    matrix = np.divide(matrix - x[column], x[column])
    matrix = matrix.flatten()
    return pd.Series(np.append(np.append(np.percentile(matrix, [0, 5, 10, 25, 50, 75, 90, 95, 100]), 
                     np.mean(matrix)), np.std(matrix)))
    
def return_dist(data, num_hours, freq , DateTime_column, volume, pair_crypto, column, columns, sort_type=True):
    result = pd.DataFrame()
    for pair in data[pair_crypto].unique():
        temp_data = data[data[pair_crypto] == pair].copy()
        temp_data.sort_values(by=[pair_crypto,DateTime_column], inplace = True, ignore_index=True, ascending=sort_type)
        temp = temp_data.head(temp_data.shape[0]-(num_hours*freq))[columns]
        temp['date_time']=temp_data[DateTime_column]
        temp['volume']=temp_data[volume]
        temp['pair']=pair
        temp["Min_"+str(num_hours)] = np.nan
        temp["P5_"+str(num_hours)] = np.nan
        temp["P10_"+str(num_hours)] = np.nan
        temp["P25_"+str(num_hours)] = np.nan
        temp["Median_"+str(num_hours)] = np.nan
        temp["P75_"+str(num_hours)] = np.nan
        temp["P90_"+str(num_hours)] = np.nan
        temp["P95_"+str(num_hours)] = np.nan
        temp["Max_"+str(num_hours)] = np.nan
        temp["Mean_"+str(num_hours)] = np.nan
        temp["STD_"+str(num_hours)] = np.nan
        temp[["Min_"+str(num_hours), "P5_"+str(num_hours), "P10_"+str(num_hours),
                  "P25_"+str(num_hours), "Median_"+str(num_hours), "P75_"+str(num_hours),
                  "P90_"+str(num_hours), "P95_"+str(num_hours), "Max_"+str(num_hours),
                  "Mean_"+str(num_hours), "STD_"+str(num_hours)]] = temp.swifter.apply(lambda x: summary_stats(x, temp_data, columns ,column, num_hours*freq), axis=1)
        result = result.append(temp)
    return result 

**Working with Train Data**

In [ ]:
#Distribution of Return in the Next 12 hour
train_data1 = return_dist(train_data,12, 4, "date_time", "volume", "pair", "close", ['low', 'high','close'])
train_data1=train_data1.reset_index(drop=True)
train_data1

Pandas Apply:   0%|          | 0/23418 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23418 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23418 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23418 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23418 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/20330 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23418 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23418 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23418 [00:00<?, ?it/s]

,low,high,close,date_time,volume,pair,Min_12,P5_12,P10_12,P25_12,Median_12,P75_12,P90_12,P95_12,Max_12,Mean_12,STD_12
0,1.16999,1.18247,1.18247,2021-04-08 00:00:00,2858686.2,ADAUSDT,-0.005987,-0.001070,0.000814,0.004590,0.008588,0.014024,0.016853,0.020144,0.030056,0.009086,0.006492
1,1.17972,1.19413,1.18306,2021-04-08 00:15:00,3479725.9,ADAUSDT,-0.006483,-0.001521,0.000848,0.004159,0.008220,0.013717,0.017050,0.019636,0.029542,0.008871,0.006484
2,1.18123,1.19108,1.18929,2021-04-08 00:30:00,1808114.1,ADAUSDT,-0.011688,-0.006480,-0.003960,-0.001085,0.003208,0.008614,0.012851,0.014963,0.024149,0.003883,0.006517
3,1.18292,1.19048,1.18681,2021-04-08 00:45:00,2074314.9,ADAUSDT,-0.009622,-0.004404,-0.001560,0.001226,0.005831,0.010857,0.015526,0.018163,0.026289,0.006354,0.006679
4,1.18120,1.18912,1.18646,2021-04-08 01:00:00,1591845.0,ADAUSDT,-0.009330,-0.003380,-0.000698,0.001787,0.006473,0.011433,0.016019,0.018622,0.026592,0.007056,0.006831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348177,0.80560,0.81650,0.80710,2021-12-08 10:45:00,10728058.0,XRPUSDT,-0.005576,0.001753,0.006889,0.019824,0.071800,0.078677,0.082902,0.084996,0.095775,0.055811,0.030701
348178,0.80260,0.81500,0.81140,2021-12-08 11:00:00,12232156.0,XRPUSDT,-0.008750,-0.003340,0.002502,0.016854,0.066120,0.072960,0.077163,0.079246,0.089968,0.051453,0.029569
348179,0.80690,0.81710,0.80860,2021-12-08 11:15:00,4826955.0,XRPUSDT,-0.005318,0.002090,0.006245,0.042945,0.069812,0.076676,0.080893,0.082983,0.093742,0.056383,0.028690
348180,0.80500,0.81260,0.80850,2021-12-08 11:30:00,4966307.0,XRPUSDT,-0.005195,0.005003,0.008077,0.047774,0.069944,0.076809,0.081027,0.083117,0.093878,0.057837,0.027505


In [ ]:
#Defining Take Profit and Stop Loss Limits 

train_data=pd.DataFrame()

for pair in train_data1['pair'].unique():
  data=train_data1[train_data1['pair']==pair]
  data['ATR']=ta.ATR(data['high'],data['low'],data['close'], timeperiod=288)
  train_data=train_data.append(data)

train_data.to_excel('stop_loss_take_profit.xlsx')

#Conservative Approach

train_data['stop_loss_a_c']=train_data['close']-1*train_data['ATR']
train_data['take_profit_a_c']=train_data['close']+2*train_data['ATR']

#Risky Approach

train_data['stop_loss_a_r']=train_data['close']-3*train_data['ATR']
train_data['take_profit_a_r']=train_data['close']+5*train_data['ATR']
train_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,low,high,close,date_time,volume,pair,Min_12,P5_12,P10_12,P25_12,...,P90_12,P95_12,Max_12,Mean_12,STD_12,ATR,stop_loss_a_c,take_profit_a_c,stop_loss_a_r,take_profit_a_r
0,1.16999,1.18247,1.18247,2021-04-08 00:00:00,2858686.2,ADAUSDT,-0.005987,-0.001070,0.000814,0.004590,...,0.016853,0.020144,0.030056,0.009086,0.006492,NaN,NaN,NaN,NaN,NaN
1,1.17972,1.19413,1.18306,2021-04-08 00:15:00,3479725.9,ADAUSDT,-0.006483,-0.001521,0.000848,0.004159,...,0.017050,0.019636,0.029542,0.008871,0.006484,NaN,NaN,NaN,NaN,NaN
2,1.18123,1.19108,1.18929,2021-04-08 00:30:00,1808114.1,ADAUSDT,-0.011688,-0.006480,-0.003960,-0.001085,...,0.012851,0.014963,0.024149,0.003883,0.006517,NaN,NaN,NaN,NaN,NaN
3,1.18292,1.19048,1.18681,2021-04-08 00:45:00,2074314.9,ADAUSDT,-0.009622,-0.004404,-0.001560,0.001226,...,0.015526,0.018163,0.026289,0.006354,0.006679,NaN,NaN,NaN,NaN,NaN
4,1.18120,1.18912,1.18646,2021-04-08 01:00:00,1591845.0,ADAUSDT,-0.009330,-0.003380,-0.000698,0.001787,...,0.016019,0.018622,0.026592,0.007056,0.006831,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348177,0.80560,0.81650,0.80710,2021-12-08 10:45:00,10728058.0,XRPUSDT,-0.005576,0.001753,0.006889,0.019824,...,0.082902,0.084996,0.095775,0.055811,0.030701,0.007826,0.799274,0.822752,0.783622,0.846230
348178,0.80260,0.81500,0.81140,2021-12-08 11:00:00,12232156.0,XRPUSDT,-0.008750,-0.003340,0.002502,0.016854,...,0.077163,0.079246,0.089968,0.051453,0.029569,0.007842,0.803558,0.827084,0.787874,0.850609
348179,0.80690,0.81710,0.80860,2021-12-08 11:15:00,4826955.0,XRPUSDT,-0.005318,0.002090,0.006245,0.042945,...,0.080893,0.082983,0.093742,0.056383,0.028690,0.007850,0.800750,0.824300,0.785050,0.847850
348180,0.80500,0.81260,0.80850,2021-12-08 11:30:00,4966307.0,XRPUSDT,-0.005195,0.005003,0.008077,0.047774,...,0.081027,0.083117,0.093878,0.057837,0.027505,0.007849,0.800651,0.824198,0.784952,0.847746


In [ ]:
#Assigning Stop Loss and Take Profit 

#Conservative Approach

train_data['sl_tp']=np.nan 

train_data_new=pd.DataFrame()

for pair in train_data['pair'].unique():
  data=train_data[train_data['pair']==pair]
  data=data.reset_index(drop=True)
  output_data=pd.DataFrame()
  for i in range(288,data.shape[0]-48):            #3-day ATR for each pair so first 288 observations will be null/nan/missing and taking 12 hour period
      if data.loc[i,'sl_tp']!=2:
        temp_data=data[i:i+48]                     #12 hour period to Analyze the Closing Prices and Compare with Stop Loss and Take Profit
        temp_data=temp_data.reset_index(drop=True)
        close_price=temp_data.loc[0,'close']
        stop_loss=temp_data.loc[0,'stop_loss_a_c']
        take_profit=temp_data.loc[0,'take_profit_a_c']
        for j in range(1,temp_data.shape[0]):
          if temp_data.loc[j,'close']>=take_profit:
            temp_data.loc[0,'sl_tp']=1
            break
          elif temp_data.loc[j,'close']<=stop_loss:
            temp_data.loc[0,'sl_tp']=0
            break
          else:
            next
        if temp_data.loc[0,'sl_tp']!=0 and temp_data.loc[0,'sl_tp']!=1:
          temp_data.loc[0,'sl_tp']=2
          data.loc[i+1:i+3,'sl_tp']=2            #1-hour Holding Period
        output_data=output_data.append(temp_data.loc[0])
      else:
        temp_data=data[i:i+48]
        temp_data=temp_data.reset_index(drop=True)
        temp_data.loc[0]=data.loc[i]
        output_data=output_data.append(temp_data.loc[0]) 
  train_data_new=train_data_new.append(output_data)

In [ ]:
#Distribution of Target Variable in Train Data
train_data=train_data_new.reset_index(drop=True)
train_data['sl_tp'].value_counts()
train_data.to_excel('train_data.xlsx')

,low,high,close,date_time,volume,pair,Min_12,P5_12,P10_12,P25_12,...,P95_12,Max_12,Mean_12,STD_12,ATR,stop_loss_a_c,take_profit_a_c,stop_loss_a_r,take_profit_a_r,sl_tp
0,1.21333,1.22670,1.22670,2021-04-11 00:00:00,3943421.8,ADAUSDT,-0.022581,-0.017689,-0.016001,-0.013080,...,0.001480,0.025320,-0.008170,0.007112,0.006197,1.220503,1.239095,1.208108,1.257687,0.0
1,1.21929,1.22694,1.22087,2021-04-11 00:15:00,2902601.6,ADAUSDT,-0.017913,-0.012998,-0.011303,-0.008367,...,0.006561,0.047007,-0.002745,0.009094,0.006202,1.214668,1.233275,1.202263,1.251882,0.0
2,1.21872,1.22587,1.22505,2021-04-11 00:30:00,2231054.2,ADAUSDT,-0.021264,-0.016365,-0.014676,-0.011751,...,0.013335,0.043435,-0.005478,0.010493,0.006206,1.218844,1.237462,1.206433,1.256079,0.0
3,1.22377,1.22888,1.22819,2021-04-11 00:45:00,3783306.4,ADAUSDT,-0.023767,-0.018880,-0.017195,-0.014277,...,0.024076,0.044741,-0.007207,0.012385,0.006202,1.221988,1.240594,1.209584,1.259200,0.0
4,1.22321,1.22836,1.22478,2021-04-11 01:00:00,1940756.4,ADAUSDT,-0.021049,-0.016149,-0.014459,-0.011533,...,0.029302,0.052434,-0.003678,0.013850,0.006198,1.218582,1.237177,1.206185,1.255772,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343137,0.80420,0.81240,0.80730,2021-12-07 22:45:00,3725952.0,XRPUSDT,-0.003964,-0.000867,0.002539,0.006132,...,0.041174,0.052768,0.016324,0.013276,0.008053,0.799247,0.823407,0.783140,0.847567,1.0
343138,0.80500,0.81030,0.80990,2021-12-07 23:00:00,2060343.0,XRPUSDT,-0.009013,-0.004075,-0.000679,0.003241,...,0.037832,0.049389,0.013094,0.013224,0.008044,0.801856,0.825988,0.785768,0.850120,1.0
343139,0.80910,0.81490,0.81150,2021-12-07 23:15:00,1701480.0,XRPUSDT,-0.010967,-0.006038,-0.003179,0.001263,...,0.035786,0.047320,0.011072,0.013236,0.008036,0.803464,0.827572,0.787392,0.851681,1.0
343140,0.81150,0.81850,0.81780,2021-12-07 23:30:00,1953232.0,XRPUSDT,-0.018586,-0.013695,-0.011335,-0.006787,...,0.027806,0.039252,0.003099,0.013272,0.008033,0.809767,0.833865,0.793702,0.857963,0.0


In [ ]:
#Distribution of Target Variable in Train Data
train_data=train_data_new.reset_index(drop=True)
train_data['sl_tp_r'].value_counts()
train_data.to_excel('train_data.xlsx')

**Working with Test Data 1**

In [ ]:
#Distribution of Return in the Next 12 hour
test_data1 = return_dist(test_data1,12, 4,"date_time", "volume", "pair", "close", ['low', 'high','close'])
test_data1=test_data1.reset_index(drop=True)
test_data1

Pandas Apply:   0%|          | 0/5904 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5904 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5904 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5904 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5904 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/5904 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5904 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5904 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5904 [00:00<?, ?it/s]

,low,high,close,date_time,volume,pair,Min_12,P5_12,P10_12,P25_12,Median_12,P75_12,P90_12,P95_12,Max_12,Mean_12,STD_12
0,50435.58,50797.76,50792.67,2021-12-09 00:00:00,446.66366,BTCUSDT,-0.037302,-0.034477,-0.031538,-0.028207,-0.021661,-0.017844,-0.008938,-0.007250,1.968788e-07,-0.021613,0.008234
1,50323.83,50792.68,50468.84,2021-12-09 00:15:00,337.65212,BTCUSDT,-0.031125,-0.028281,-0.025425,-0.022187,-0.015948,-0.011848,-0.003705,-0.001369,3.996922e-03,-0.015827,0.007997
2,50399.22,50612.48,50570.83,2021-12-09 00:30:00,203.23244,BTCUSDT,-0.033079,-0.030241,-0.027393,-0.024236,-0.018140,-0.014212,-0.006187,-0.003728,1.972085e-03,-0.018313,0.007649
3,50388.02,50670.56,50399.83,2021-12-09 00:45:00,177.59336,BTCUSDT,-0.029798,-0.026951,-0.024093,-0.021120,-0.015224,-0.011440,-0.004011,-0.002209,1.663696e-03,-0.015436,0.007303
4,50265.22,50483.68,50282.17,2021-12-09 01:00:00,241.82873,BTCUSDT,-0.027528,-0.024674,-0.022064,-0.019245,-0.013156,-0.009280,-0.003784,-0.000035,2.995694e-03,-0.013591,0.007097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88555,18.16,18.27,18.18,2022-02-08 10:45:00,44626.80000,LINKUSDT,-0.025303,-0.020270,-0.017987,-0.012239,-0.006326,0.000688,0.007151,0.009901,1.155116e-02,-0.005565,0.009176
88556,18.15,18.30,18.17,2022-02-08 11:00:00,47602.61000,LINKUSDT,-0.024766,-0.019730,-0.017446,-0.011695,-0.005779,0.002201,0.009026,0.010457,1.541002e-02,-0.004777,0.009501
88557,18.12,18.24,18.16,2022-02-08 11:15:00,40417.37000,LINKUSDT,-0.024229,-0.019191,-0.016905,-0.011151,-0.005231,0.003304,0.010463,0.012115,2.092511e-02,-0.003847,0.010051
88558,18.12,18.22,18.19,2022-02-08 11:30:00,20205.69000,LINKUSDT,-0.025838,-0.020808,-0.018527,-0.012782,-0.006872,0.002886,0.009346,0.013332,1.979109e-02,-0.005123,0.010512


In [ ]:
#Defining Take Profit and Stop Loss Limits 

test_data=pd.DataFrame()

for pair in test_data1['pair'].unique():
  data=test_data1[test_data1['pair']==pair]
  data['ATR']=ta.ATR(data['high'],data['low'],data['close'], timeperiod=288)
  test_data=test_data.append(data)

test_data.to_excel('stop_loss_take_profit_test1.xlsx')

#Conservative Approach

test_data['stop_loss_a_c']=test_data['close']-1*test_data['ATR']
test_data['take_profit_a_c']=test_data['close']+2*test_data['ATR']

#Risky Approach

test_data['stop_loss_a_r']=test_data['close']-3*test_data['ATR']
test_data['take_profit_a_r']=test_data['close']+5*test_data['ATR']
test_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,low,high,close,date_time,volume,pair,Min_12,P5_12,P10_12,P25_12,...,P90_12,P95_12,Max_12,Mean_12,STD_12,ATR,stop_loss_a_c,take_profit_a_c,stop_loss_a_r,take_profit_a_r
0,50435.58,50797.76,50792.67,2021-12-09 00:00:00,446.66366,BTCUSDT,-0.037302,-0.034477,-0.031538,-0.028207,...,-0.008938,-0.007250,1.968788e-07,-0.021613,0.008234,NaN,NaN,NaN,NaN,NaN
1,50323.83,50792.68,50468.84,2021-12-09 00:15:00,337.65212,BTCUSDT,-0.031125,-0.028281,-0.025425,-0.022187,...,-0.003705,-0.001369,3.996922e-03,-0.015827,0.007997,NaN,NaN,NaN,NaN,NaN
2,50399.22,50612.48,50570.83,2021-12-09 00:30:00,203.23244,BTCUSDT,-0.033079,-0.030241,-0.027393,-0.024236,...,-0.006187,-0.003728,1.972085e-03,-0.018313,0.007649,NaN,NaN,NaN,NaN,NaN
3,50388.02,50670.56,50399.83,2021-12-09 00:45:00,177.59336,BTCUSDT,-0.029798,-0.026951,-0.024093,-0.021120,...,-0.004011,-0.002209,1.663696e-03,-0.015436,0.007303,NaN,NaN,NaN,NaN,NaN
4,50265.22,50483.68,50282.17,2021-12-09 01:00:00,241.82873,BTCUSDT,-0.027528,-0.024674,-0.022064,-0.019245,...,-0.003784,-0.000035,2.995694e-03,-0.013591,0.007097,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88555,18.16,18.27,18.18,2022-02-08 10:45:00,44626.80000,LINKUSDT,-0.025303,-0.020270,-0.017987,-0.012239,...,0.007151,0.009901,1.155116e-02,-0.005565,0.009176,0.144878,18.035122,18.469757,17.745365,18.904392
88556,18.15,18.30,18.17,2022-02-08 11:00:00,47602.61000,LINKUSDT,-0.024766,-0.019730,-0.017446,-0.011695,...,0.009026,0.010457,1.541002e-02,-0.004777,0.009501,0.144896,18.025104,18.459792,17.735312,18.894481
88557,18.12,18.24,18.16,2022-02-08 11:15:00,40417.37000,LINKUSDT,-0.024229,-0.019191,-0.016905,-0.011151,...,0.010463,0.012115,2.092511e-02,-0.003847,0.010051,0.144810,18.015190,18.449619,17.725571,18.884048
88558,18.12,18.22,18.19,2022-02-08 11:30:00,20205.69000,LINKUSDT,-0.025838,-0.020808,-0.018527,-0.012782,...,0.009346,0.013332,1.979109e-02,-0.005123,0.010512,0.144654,18.045346,18.479308,17.756038,18.913271


In [ ]:
#Assigning Stop Loss and Take Profit 

#Conservative Approach

test_data['sl_tp']=np.nan 

test_data_new=pd.DataFrame()

for pair in test_data['pair'].unique():
  data=test_data[test_data['pair']==pair]
  data=data.reset_index(drop=True)
  output_data=pd.DataFrame()
  for i in range(288,data.shape[0]-48):            #3-day ATR for each pair so first 288 observations will be null/nan/missing
      if data.loc[i,'sl_tp']!=2:
        temp_data=data[i:i+48]                     #12 hour period to Analyze the Closing Prices and Compare with Stop Loss and Take Profit
        temp_data=temp_data.reset_index(drop=True)
        close_price=temp_data.loc[0,'close']
        stop_loss=temp_data.loc[0,'stop_loss_a_c']
        take_profit=temp_data.loc[0,'take_profit_a_c']
        for j in range(1,temp_data.shape[0]):
          if temp_data.loc[j,'close']>=take_profit:
            temp_data.loc[0,'sl_tp']=1
            break
          elif temp_data.loc[j,'close']<=stop_loss:
            temp_data.loc[0,'sl_tp']=0
            break
          else:
            next
        if temp_data.loc[0,'sl_tp']!=0 and temp_data.loc[0,'sl_tp']!=1:
          temp_data.loc[0,'sl_tp']=2
          data.loc[i+1:i+3,'sl_tp']=2            #1-hour Holding Period
        output_data=output_data.append(temp_data.loc[0])
      else:
        temp_data=data[i:i+48]
        temp_data=temp_data.reset_index(drop=True)
        temp_data.loc[0]=data.loc[i]
        output_data=output_data.append(temp_data.loc[0]) 
  test_data_new=test_data_new.append(output_data)

In [ ]:
#Risky Approach

test_data['sl_tp_r']=np.nan 

test_data_new=pd.DataFrame()

for pair in test_data['pair'].unique():
  data=test_data[test_data['pair']==pair]
  data=data.reset_index(drop=True)
  output_data=pd.DataFrame()
  for i in range(288,data.shape[0]-48):            #3-day ATR for each pair so first 288 observations will be null/nan/missing
      if data.loc[i,'sl_tp_r']!=2:
        temp_data=data[i:i+48]                     #12 hour period to Analyze the Closing Prices and Compare with Stop Loss and Take Profit
        temp_data=temp_data.reset_index(drop=True)
        close_price=temp_data.loc[0,'close']
        stop_loss=temp_data.loc[0,'stop_loss_a_r']
        take_profit=temp_data.loc[0,'take_profit_a_r']
        for j in range(1,temp_data.shape[0]):
          if temp_data.loc[j,'close']>=take_profit:
            temp_data.loc[0,'sl_tp_r']=1
            break
          elif temp_data.loc[j,'close']<=stop_loss:
            temp_data.loc[0,'sl_tp_r']=0
            break
          else:
            next
        if temp_data.loc[0,'sl_tp_r']!=0 and temp_data.loc[0,'sl_tp_r']!=1:
          temp_data.loc[0,'sl_tp_r']=2
          data.loc[i+1:i+3,'sl_tp_r']=2            #1-hour Holding Period
        output_data=output_data.append(temp_data.loc[0])
      else:
        temp_data=data[i:i+48]
        temp_data=temp_data.reset_index(drop=True)
        temp_data.loc[0]=data.loc[i]
        output_data=output_data.append(temp_data.loc[0]) 
  test_data_new=test_data_new.append(output_data)

In [ ]:
#Distribution of Target Variable in Train Data
test_data=test_data_new.reset_index(drop=True)
display(test_data['sl_tp'].value_counts())
display(test_data['sl_tp_r'].value_counts())
test_data.to_excel('test_data1.xlsx')

0.0    48261
1.0    26883
2.0     3336
Name: sl_tp, dtype: int64

2.0    39555
0.0    26933
1.0    11992
Name: sl_tp_r, dtype: int64

 **Working with Test Data 2**

In [ ]:
#Distribution of Return in the Next 12 hour
test_data2 = return_dist(test_data2,12, 4,"date_time", "volume", "pair", "close", ['low', 'high','close'])
test_data2=test_data2.reset_index(drop=True)
test_data2

Pandas Apply:   0%|          | 0/5137 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5137 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5137 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5137 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5137 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/5137 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5137 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5137 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5137 [00:00<?, ?it/s]

,low,high,close,date_time,volume,pair,Min_12,P5_12,P10_12,P25_12,Median_12,P75_12,P90_12,P95_12,Max_12,Mean_12,STD_12
0,44000.00,44146.20,44115.17,2022-02-09 00:00:00,394.10543,BTCUSDT,-0.022606,-0.019554,-0.018428,-0.016216,-0.012542,-0.008788,-0.002309,0.000815,0.005139,-0.011659,0.006089
1,43971.67,44193.42,44016.59,2022-02-09 00:15:00,336.07521,BTCUSDT,-0.020417,-0.017358,-0.016229,-0.014013,-0.010331,-0.006568,-0.001449,0.002789,0.007390,-0.009563,0.005947
2,43958.70,44266.76,44257.75,2022-02-09 00:30:00,501.64743,BTCUSDT,-0.025754,-0.022712,-0.021590,-0.019386,-0.015724,-0.011981,-0.007285,-0.002675,0.001901,-0.014977,0.005913
3,44162.02,44341.88,44243.99,2022-02-09 00:45:00,303.39698,BTCUSDT,-0.025451,-0.022409,-0.021286,-0.019081,-0.015417,-0.011674,-0.007074,-0.003003,0.002379,-0.014721,0.005818
4,44087.55,44247.54,44115.31,2022-02-09 01:00:00,347.37589,BTCUSDT,-0.022609,-0.019557,-0.018431,-0.016220,-0.012546,-0.008791,-0.004177,0.000294,0.008072,-0.011783,0.006016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77050,17.62,17.74,17.67,2022-04-03 11:00:00,26246.80000,LINKUSDT,-0.001698,0.001698,0.003396,0.007357,0.014997,0.022637,0.027731,0.030956,0.036786,0.015504,0.009533
77051,17.65,17.77,17.77,2022-04-03 11:15:00,32739.57000,LINKUSDT,-0.007316,-0.003855,-0.002082,0.002251,0.010129,0.017164,0.022510,0.025155,0.030951,0.010270,0.009442
77052,17.77,17.85,17.85,2022-04-03 11:30:00,31835.63000,LINKUSDT,-0.011765,-0.008319,-0.006555,-0.001821,0.006162,0.013585,0.018319,0.020644,0.026331,0.006077,0.009434
77053,17.76,17.85,17.79,2022-04-03 11:45:00,28958.42000,LINKUSDT,-0.008432,-0.004975,-0.003204,0.001686,0.010118,0.017426,0.021754,0.024087,0.029792,0.009782,0.009413


In [ ]:
#Defining Take Profit and Stop Loss Limits 

test_data1=pd.DataFrame()

for pair in test_data2['pair'].unique():
  data=test_data2[test_data2['pair']==pair]
  data['ATR']=ta.ATR(data['high'],data['low'],data['close'], timeperiod=288)
  test_data1=test_data1.append(data)

test_data1.to_excel('stop_loss_take_profit_test2.xlsx')

#Conservative Approach

test_data1['stop_loss_a_c']=test_data1['close']-1*test_data1['ATR']
test_data1['take_profit_a_c']=test_data1['close']+2*test_data1['ATR']

#Risky Approach

test_data1['stop_loss_a_r']=test_data1['close']-3*test_data1['ATR']
test_data1['take_profit_a_r']=test_data1['close']+5*test_data1['ATR']
test_data1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,low,high,close,date_time,volume,pair,Min_12,P5_12,P10_12,P25_12,...,P90_12,P95_12,Max_12,Mean_12,STD_12,ATR,stop_loss_a_c,take_profit_a_c,stop_loss_a_r,take_profit_a_r
0,44000.00,44146.20,44115.17,2022-02-09 00:00:00,394.10543,BTCUSDT,-0.022606,-0.019554,-0.018428,-0.016216,...,-0.002309,0.000815,0.005139,-0.011659,0.006089,NaN,NaN,NaN,NaN,NaN
1,43971.67,44193.42,44016.59,2022-02-09 00:15:00,336.07521,BTCUSDT,-0.020417,-0.017358,-0.016229,-0.014013,...,-0.001449,0.002789,0.007390,-0.009563,0.005947,NaN,NaN,NaN,NaN,NaN
2,43958.70,44266.76,44257.75,2022-02-09 00:30:00,501.64743,BTCUSDT,-0.025754,-0.022712,-0.021590,-0.019386,...,-0.007285,-0.002675,0.001901,-0.014977,0.005913,NaN,NaN,NaN,NaN,NaN
3,44162.02,44341.88,44243.99,2022-02-09 00:45:00,303.39698,BTCUSDT,-0.025451,-0.022409,-0.021286,-0.019081,...,-0.007074,-0.003003,0.002379,-0.014721,0.005818,NaN,NaN,NaN,NaN,NaN
4,44087.55,44247.54,44115.31,2022-02-09 01:00:00,347.37589,BTCUSDT,-0.022609,-0.019557,-0.018431,-0.016220,...,-0.004177,0.000294,0.008072,-0.011783,0.006016,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77050,17.62,17.74,17.67,2022-04-03 11:00:00,26246.80000,LINKUSDT,-0.001698,0.001698,0.003396,0.007357,...,0.027731,0.030956,0.036786,0.015504,0.009533,0.112447,17.557553,17.894894,17.332659,18.232235
77051,17.65,17.77,17.77,2022-04-03 11:15:00,32739.57000,LINKUSDT,-0.007316,-0.003855,-0.002082,0.002251,...,0.022510,0.025155,0.030951,0.010270,0.009442,0.112473,17.657527,17.994946,17.432580,18.332366
77052,17.77,17.85,17.85,2022-04-03 11:30:00,31835.63000,LINKUSDT,-0.011765,-0.008319,-0.006555,-0.001821,...,0.018319,0.020644,0.026331,0.006077,0.009434,0.112360,17.737640,18.074721,17.512919,18.411802
77053,17.76,17.85,17.79,2022-04-03 11:45:00,28958.42000,LINKUSDT,-0.008432,-0.004975,-0.003204,0.001686,...,0.021754,0.024087,0.029792,0.009782,0.009413,0.112283,17.677717,18.014566,17.453151,18.351414


In [ ]:
#Assigning Stop Loss and Take Profit 

#Conservative Approach

test_data1['sl_tp']=np.nan 

test_data1_new=pd.DataFrame()

for pair in test_data1['pair'].unique():
  data=test_data1[test_data1['pair']==pair]
  data=data.reset_index(drop=True)
  output_data=pd.DataFrame()
  for i in range(288,data.shape[0]-48):            #3-day ATR for each pair so first 288 observations will be null/nan/missing
      if data.loc[i,'sl_tp']!=2:
        temp_data=data[i:i+48]                     #12 hour period to Analyze the Closing Prices and Compare with Stop Loss and Take Profit
        temp_data=temp_data.reset_index(drop=True)
        close_price=temp_data.loc[0,'close']
        stop_loss=temp_data.loc[0,'stop_loss_a_c']
        take_profit=temp_data.loc[0,'take_profit_a_c']
        for j in range(1,temp_data.shape[0]):
          if temp_data.loc[j,'close']>=take_profit:
            temp_data.loc[0,'sl_tp']=1
            break
          elif temp_data.loc[j,'close']<=stop_loss:
            temp_data.loc[0,'sl_tp']=0
            break
          else:
            next
        if temp_data.loc[0,'sl_tp']!=0 and temp_data.loc[0,'sl_tp']!=1:
          temp_data.loc[0,'sl_tp']=2
          data.loc[i+1:i+3,'sl_tp']=2            #1-hour Holding Period
        output_data=output_data.append(temp_data.loc[0])
      else:
        temp_data=data[i:i+48]
        temp_data=temp_data.reset_index(drop=True)
        temp_data.loc[0]=data.loc[i]
        output_data=output_data.append(temp_data.loc[0]) 
  test_data1_new=test_data1_new.append(output_data)

In [ ]:
#Risky Approach

test_data1['sl_tp_r']=np.nan 

test_data1_new=pd.DataFrame()

for pair in test_data1['pair'].unique():
  data=test_data1[test_data1['pair']==pair]
  data=data.reset_index(drop=True)
  output_data=pd.DataFrame()
  for i in range(288,data.shape[0]-48):            #3-day ATR for each pair so first 288 observations will be null/nan/missing
      if data.loc[i,'sl_tp_r']!=2:
        temp_data=data[i:i+48]                     #12 hour period to Analyze the Closing Prices and Compare with Stop Loss and Take Profit
        temp_data=temp_data.reset_index(drop=True)
        close_price=temp_data.loc[0,'close']
        stop_loss=temp_data.loc[0,'stop_loss_a_r']
        take_profit=temp_data.loc[0,'take_profit_a_r']
        for j in range(1,temp_data.shape[0]):
          if temp_data.loc[j,'close']>=take_profit:
            temp_data.loc[0,'sl_tp_r']=1
            break
          elif temp_data.loc[j,'close']<=stop_loss:
            temp_data.loc[0,'sl_tp_r']=0
            break
          else:
            next
        if temp_data.loc[0,'sl_tp_r']!=0 and temp_data.loc[0,'sl_tp_r']!=1:
          temp_data.loc[0,'sl_tp_r']=2
          data.loc[i+1:i+3,'sl_tp_r']=2            #1-hour Holding Period
        output_data=output_data.append(temp_data.loc[0])
      else:
        temp_data=data[i:i+48]
        temp_data=temp_data.reset_index(drop=True)
        temp_data.loc[0]=data.loc[i]
        output_data=output_data.append(temp_data.loc[0]) 
  test_data1_new=test_data1_new.append(output_data)

In [ ]:
#Distribution of Target Variable in Train Data
test_data1=test_data1_new.reset_index(drop=True)
test_data1['sl_tp'].value_counts()
test_data1['sl_tp_r'].value_counts()
test_data1.to_excel('test_data2.xlsx')
test_data2

,low,high,close,date_time,volume,pair,Min_12,P5_12,P10_12,P25_12,Median_12,P75_12,P90_12,P95_12,Max_12,Mean_12,STD_12
0,44000.00,44146.20,44115.17,2022-02-09 00:00:00,394.10543,BTCUSDT,-0.022606,-0.019554,-0.018428,-0.016216,-0.012542,-0.008788,-0.002309,0.000815,0.005139,-0.011659,0.006089
1,43971.67,44193.42,44016.59,2022-02-09 00:15:00,336.07521,BTCUSDT,-0.020417,-0.017358,-0.016229,-0.014013,-0.010331,-0.006568,-0.001449,0.002789,0.007390,-0.009563,0.005947
2,43958.70,44266.76,44257.75,2022-02-09 00:30:00,501.64743,BTCUSDT,-0.025754,-0.022712,-0.021590,-0.019386,-0.015724,-0.011981,-0.007285,-0.002675,0.001901,-0.014977,0.005913
3,44162.02,44341.88,44243.99,2022-02-09 00:45:00,303.39698,BTCUSDT,-0.025451,-0.022409,-0.021286,-0.019081,-0.015417,-0.011674,-0.007074,-0.003003,0.002379,-0.014721,0.005818
4,44087.55,44247.54,44115.31,2022-02-09 01:00:00,347.37589,BTCUSDT,-0.022609,-0.019557,-0.018431,-0.016220,-0.012546,-0.008791,-0.004177,0.000294,0.008072,-0.011783,0.006016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77050,17.62,17.74,17.67,2022-04-03 11:00:00,26246.80000,LINKUSDT,-0.001698,0.001698,0.003396,0.007357,0.014997,0.022637,0.027731,0.030956,0.036786,0.015504,0.009533
77051,17.65,17.77,17.77,2022-04-03 11:15:00,32739.57000,LINKUSDT,-0.007316,-0.003855,-0.002082,0.002251,0.010129,0.017164,0.022510,0.025155,0.030951,0.010270,0.009442
77052,17.77,17.85,17.85,2022-04-03 11:30:00,31835.63000,LINKUSDT,-0.011765,-0.008319,-0.006555,-0.001821,0.006162,0.013585,0.018319,0.020644,0.026331,0.006077,0.009434
77053,17.76,17.85,17.79,2022-04-03 11:45:00,28958.42000,LINKUSDT,-0.008432,-0.004975,-0.003204,0.001686,0.010118,0.017426,0.021754,0.024087,0.029792,0.009782,0.009413


**Assigning Stop Loss and Take Profit-Training Data (Risky Approach)**

**Train Data**

In [ ]:
train_data=pd.read_excel(r'train_data.xlsx')

train_data=train_data.drop(['Unnamed: 0'],axis=1)

In [ ]:
train_data.head()

,low,high,close,date_time,volume,pair,Min_12,P5_12,P10_12,P25_12,...,P95_12,Max_12,Mean_12,STD_12,ATR,stop_loss_a_c,take_profit_a_c,stop_loss_a_r,take_profit_a_r,sl_tp
0,1.21333,1.22670,1.22670,2021-04-11 00:00:00,3943421.8,ADAUSDT,-0.022581,-0.017689,-0.016001,-0.013080,...,0.001480,0.025320,-0.008170,0.007112,0.006197,1.220503,1.239095,1.208108,1.257687,0
1,1.21929,1.22694,1.22087,2021-04-11 00:15:00,2902601.6,ADAUSDT,-0.017913,-0.012998,-0.011303,-0.008367,...,0.006561,0.047007,-0.002745,0.009094,0.006202,1.214668,1.233275,1.202263,1.251882,0
2,1.21872,1.22587,1.22505,2021-04-11 00:30:00,2231054.2,ADAUSDT,-0.021264,-0.016365,-0.014676,-0.011751,...,0.013335,0.043435,-0.005478,0.010493,0.006206,1.218844,1.237462,1.206433,1.256079,0
3,1.22377,1.22888,1.22819,2021-04-11 00:45:00,3783306.4,ADAUSDT,-0.023767,-0.018880,-0.017195,-0.014277,...,0.024076,0.044741,-0.007207,0.012385,0.006202,1.221988,1.240594,1.209584,1.259200,0
4,1.22321,1.22836,1.22478,2021-04-11 01:00:00,1940756.4,ADAUSDT,-0.021049,-0.016149,-0.014459,-0.011533,...,0.029302,0.052434,-0.003678,0.013850,0.006198,1.218582,1.237177,1.206185,1.255772,0


In [ ]:
train_data['sl_tp_r']=np.nan 

train_data_new=pd.DataFrame()

for pair in train_data['pair'].unique():
  data=train_data[train_data['pair']==pair]
  data=data.reset_index(drop=True)
  output_data=pd.DataFrame()
  for i in range(288,data.shape[0]-48):            #3-day ATR for each pair so first 288 observations will be null/nan/missing and taking 12 hour period
      if data.loc[i,'sl_tp_r']!=2:
        temp_data=data[i:i+48]                     #12 hour period to Analyze the Closing Prices and Compare with Stop Loss and Take Profit
        temp_data=temp_data.reset_index(drop=True)
        close_price=temp_data.loc[0,'close']
        stop_loss=temp_data.loc[0,'stop_loss_a_r']
        take_profit=temp_data.loc[0,'take_profit_a_r']
        for j in range(1,temp_data.shape[0]):
          if temp_data.loc[j,'close']>=take_profit:
            temp_data.loc[0,'sl_tp_r']=1
            break
          elif temp_data.loc[j,'close']<=stop_loss:
            temp_data.loc[0,'sl_tp_r']=0
            break
          else:
            next
        if temp_data.loc[0,'sl_tp_r']!=0 and temp_data.loc[0,'sl_tp_r']!=1:
          temp_data.loc[0,'sl_tp_r']=2
          data.loc[i+1:i+3,'sl_tp_r']=2            #1-hour Holding Period
        output_data=output_data.append(temp_data.loc[0])
      else:
        temp_data=data[i:i+48]
        temp_data=temp_data.reset_index(drop=True)
        temp_data.loc[0]=data.loc[i]
        output_data=output_data.append(temp_data.loc[0]) 
  train_data_new=train_data_new.append(output_data)

In [ ]:
#Distribution of Target Variable in Train Data
train_data=train_data_new.reset_index(drop=True)
train_data['sl_tp_r'].value_counts()
train_data.to_excel('train_data.xlsx')

**Train Data Feature Engineering**

In [ ]:
train_data=pd.read_excel(r'train_data.xlsx')

train_data=train_data.drop(['Unnamed: 0'],axis=1)

**Feature Engineering-Constructing Technical Indicators**

In [ ]:
#(Pooled Data-Estimators)

train_data_final=pd.DataFrame()

for pair in train_data['pair'].unique():
  train_data_n=train_data[train_data['pair']==pair]
#Moving Averages of Average Returns

  train_data_n['ma_4h']=ta.SMA(train_data_n['Mean_12'], 16)                #4 Hour Moving Average
  train_data_n['ma_8h']=ta.SMA(train_data_n['Mean_12'], 32)                #8 Hour Moving Average  
  train_data_n['ma_12h']=ta.SMA(train_data_n['Mean_12'], 48)               #12 Hour Moving Average
  train_data_n['ma_24h']=ta.SMA(train_data_n['Mean_12'], 96)               #1 Day Moving Average
  train_data_n['ma_9']=ta.SMA(train_data_n['Mean_12'], 864)                #9 Day Moving Average
  train_data_n['ma_20']=ta.SMA(train_data_n['Mean_12'], 1920)              #20 Day Moving Average 
  train_data_n['ma_cross']=train_data_n['ma_9']-train_data_n['ma_20']      #Moving Average Cross Over

  #Weighted Moving Averages of Average Returns

  train_data_n['wma_4h']=ta.WMA(train_data_n['Mean_12'], 16)                #4 Hour W-Moving Average
  train_data_n['wma_8h']=ta.WMA(train_data_n['Mean_12'], 32)                #8 Hour W-Moving Average  
  train_data_n['wma_12h']=ta.WMA(train_data_n['Mean_12'], 48)               #12 Hour W-Moving Average
  train_data_n['wma_24h']=ta.WMA(train_data_n['Mean_12'], 96)               #1 Day W-Moving Average
  train_data_n['wma_9']=ta.WMA(train_data_n['Mean_12'], 864)                #9 Day W-Moving Average
  train_data_n['wma_20']=ta.WMA(train_data_n['Mean_12'], 1920)              #20 Day W-Moving Average 
  train_data_n['wma_cross']=train_data_n['wma_9']-train_data_n['wma_20']      #W-Moving Average Cross Over

  #Exponential Moving Averages of Average Returns
  train_data_n['ema_4h']=ta.EMA(train_data_n['Mean_12'], 16)                #4 Hour Moving Average
  train_data_n['ema_8h']=ta.EMA(train_data_n['Mean_12'], 32)                #8 Hour Moving Average  
  train_data_n['ema_12h']=ta.EMA(train_data_n['Mean_12'], 48)               #12 Hour Moving Average
  train_data_n['ema_24h']=ta.EMA(train_data_n['Mean_12'], 96)               #1 Day Moving Average
  train_data_n['ema_12']=ta.EMA(train_data_n['Mean_12'], 1152)              #12 Day Exponential Moving Average
  train_data_n['ema_26']=ta.EMA(train_data_n['Mean_12'], 2496)              #26 Day Exponential Moving Average

  #Standard Deviation of Returns 
  train_data_n['std_24h']=ta.STDDEV(train_data_n['Mean_12'], 96)            #1 Day Standard Deviation
  train_data_n['std_10']=ta.STDDEV(train_data_n['Mean_12'], 960)            #10 Day Standard Deviation
  train_data_n['std_10']=ta.STDDEV(train_data_n['Mean_12'], 1920)           #20 Day Standard Deviation

  #Normalized Average True Range 
  train_data_n['NATR']=ta.NATR(train_data_n['high'], train_data_n['low'],train_data_n['close'],288)         #3 Day Normalized Average True Range

  #Rate of Change 
  train_data_n['roc_20']=ta.ROC(train_data_n['close'], 20)                         #20 Period Closing Price Rate of Change

  #Percentage Price Oscillator
  train_data_n['ppo_12_26']=ta.PPO(train_data_n['close'], 12,26)                   #PPO using 12 and 26 period moving averages

  #Relative Strength Index
  train_data_n['rsi_10']=ta.RSI(train_data_n['close'],960)

  #RSI Overbought and Oversold 
  train_data_n['rsi_ob_os']=np.where((train_data_n['rsi_10'] >=30) & (train_data_n['rsi_10'] <=70),2,
                                  np.where(train_data_n['rsi_10']<30,0,
                                  np.where(train_data_n['rsi_10']>70,1,np.nan)))    #Overbought (1)/Oversold (0)/Hold (2)

  #Average Directional Index

  train_data_n['adx']=ta.ADX(train_data_n['high'], train_data_n['low'], train_data_n['close'], 672)              #7 Day ADX
  train_data_n['adx_14']=ta.ADX(train_data_n['high'], train_data_n['low'], train_data_n['close'], 1344)          #14 Day ADX

  #Strength of Price Trends using ADX
  train_data_n['price_trend_adx']=np.where((train_data_n['adx_14']>=20) & (train_data_n['adx_14']<=25),2,
                                np.where(train_data_n['adx_14']<20,0,
                                np.where(train_data_n['adx_14']>25,1,np.nan)))   #Strong Price Trend (1)/Weak Price Trend (0)/Absence of Price Trend (2)

  #Aroon Indicator
  train_data_n['aroon_down'], train_data_n['aroon_up']=ta.AROON(train_data_n['high'], train_data_n['close'], 26)           #26 Period Aroon Indicator

  #Aroon Oscillator
  train_data_n['aroon_osc']=ta.AROONOSC(train_data_n['high'], train_data_n['low'], 26)                                      #26 Period Aroon Oscillator

  #Trade Signals-Aroon Oscillator (Upward and Downward Trend Prices)
  train_data_n['price_trend_aroon']=np.where((train_data_n['aroon_osc']>=-70) &(train_data_n['aroon_osc']<=70),
                                        2,np.where(train_data_n['aroon_osc']<-70,0,np.where(train_data_n['aroon_osc']>70,1,np.nan)))   #Strong Price Trend (1)/Weak Price Trend (0)/Absence of Price Trend (2)

  #Bolllinger Bands 
  train_data_n['upperBB'], train_data_n['middleBB'], train_data_n['lowerBB'] = ta.BBANDS(train_data_n['Mean_12'], 2, 2, 20) #20 Period Bollinger Bands of Average Returns with 2 Standard Deviations 

  #14-Day Fast-Stochastic Oscillator Indicator (%K) and 3-Day Moving Average of Fast-Stochastic Oscillator Indicator (%D)
  train_data_n['fastk'], train_data_n['fastd'] = ta.STOCHF(train_data_n['high'], train_data_n['low'], train_data_n['close'], fastk_period=5, fastd_period=3, fastd_matype=0)

  #Overbought and Oversold Currency Signal based on Fast-Stochastic Oscillator
  train_data_n['kfast_ob_os']=np.where(train_data_n['fastk']>80, 1, np.where(train_data_n['fastk']<20,0,2))  #Overbought (1)/Oversold (0)/Hold (2)

  #Normalized On-Balance Volume
  train_data_n['obv']=ta.OBV(train_data_n['close'], train_data_n['volume'])
  train_data_n['volume_cumulative']=np.cumsum(train_data_n['volume'])                       #Divide Current OBV with Cumulative Volume
  train_data_n['obv_norm']=(train_data_n['obv']/train_data_n['volume_cumulative'])*100      #Ranges between -100 and 100 

  train_data_final=train_data_final.append(train_data_n)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
#Summary Statistics of Features

#Numeric Variables
train_data_final.describe([0.1,0.25,0.5,0.75,.99]).transpose().to_csv('summary_stats.csv')

#Missing Values
((train_data_final.isna().sum())/len(train_data_final)).transpose().to_csv('missing.csv')

#Categorical Variables
display(train_data_final['rsi_ob_os'].value_counts()/len(train_data_final))

display(train_data_final['price_trend_adx'].value_counts()/len(train_data_final))

display(train_data_final['price_trend_aroon'].value_counts()/len(train_data_final))

display(train_data_final['kfast_ob_os'].value_counts()/len(train_data_final))

2.0    0.957409
Name: rsi_ob_os, dtype: float64

0.0    0.88079
Name: price_trend_adx, dtype: float64

2.0    0.701300
1.0    0.156947
0.0    0.140600
Name: price_trend_aroon, dtype: float64

2    0.577308
1    0.224855
0    0.197837
Name: kfast_ob_os, dtype: float64

**Missing Values Handling and Features Exclusion**

In [ ]:
#Drop Missing Observations in Data
train_data_final=train_data_final.dropna(axis=0)
len(train_data_final)

#Drop the Categorical Variables with no Variation

train_data_final=train_data_final.drop(['rsi_ob_os','price_trend_adx'],axis=1)

#Drop Unwanted Features

train_data_final=train_data_final.drop(['low','high','close','date_time','volume','ATR','stop_loss_a_c','take_profit_a_c','stop_loss_a_r','take_profit_a_r'],axis=1)

pair                 0.0
Min_12               0.0
P5_12                0.0
P10_12               0.0
P25_12               0.0
Median_12            0.0
P75_12               0.0
P90_12               0.0
P95_12               0.0
Max_12               0.0
Mean_12              0.0
STD_12               0.0
sl_tp                0.0
sl_tp_r              0.0
ma_4h                0.0
ma_8h                0.0
ma_12h               0.0
ma_24h               0.0
ma_9                 0.0
ma_20                0.0
ma_cross             0.0
wma_4h               0.0
wma_8h               0.0
wma_12h              0.0
wma_24h              0.0
wma_9                0.0
wma_20               0.0
wma_cross            0.0
ema_4h               0.0
ema_8h               0.0
ema_12h              0.0
ema_24h              0.0
ema_12               0.0
ema_26               0.0
std_24h              0.0
std_10               0.0
NATR                 0.0
roc_20               0.0
ppo_12_26            0.0
rsi_10               0.0


In [ ]:
#Export the Final Data set

train_data_final.to_excel("train_data_final.xlsx")

In [ ]:
#Save Final Dataset

train_data_final.to_excel('train_data_final.xlsx')

**Test Data 1 Feature Engineering**

In [ ]:
test_data1=pd.read_excel(r'test_data1.xlsx')

test_data1=test_data1.drop(['Unnamed: 0'],axis=1)

**Feature Engineering**

In [ ]:
#(Pooled Data-Estimators)

test_data1_final=pd.DataFrame()

for pair in test_data1['pair'].unique():
  test_data_n=test_data1[test_data1['pair']==pair]
#Moving Averages of Average Returns

  test_data_n['ma_4h']=ta.SMA(test_data_n['Mean_12'], 16)                #4 Hour Moving Average
  test_data_n['ma_8h']=ta.SMA(test_data_n['Mean_12'], 32)                #8 Hour Moving Average  
  test_data_n['ma_12h']=ta.SMA(test_data_n['Mean_12'], 48)               #12 Hour Moving Average
  test_data_n['ma_24h']=ta.SMA(test_data_n['Mean_12'], 96)               #1 Day Moving Average
  test_data_n['ma_9']=ta.SMA(test_data_n['Mean_12'], 864)                #9 Day Moving Average
  test_data_n['ma_20']=ta.SMA(test_data_n['Mean_12'], 1920)              #20 Day Moving Average 
  test_data_n['ma_cross']=test_data_n['ma_9']-test_data_n['ma_20']      #Moving Average Cross Over

  #Weighted Moving Averages of Average Returns

  test_data_n['wma_4h']=ta.WMA(test_data_n['Mean_12'], 16)                #4 Hour W-Moving Average
  test_data_n['wma_8h']=ta.WMA(test_data_n['Mean_12'], 32)                #8 Hour W-Moving Average  
  test_data_n['wma_12h']=ta.WMA(test_data_n['Mean_12'], 48)               #12 Hour W-Moving Average
  test_data_n['wma_24h']=ta.WMA(test_data_n['Mean_12'], 96)               #1 Day W-Moving Average
  test_data_n['wma_9']=ta.WMA(test_data_n['Mean_12'], 864)                #9 Day W-Moving Average
  test_data_n['wma_20']=ta.WMA(test_data_n['Mean_12'], 1920)              #20 Day W-Moving Average 
  test_data_n['wma_cross']=test_data_n['wma_9']-test_data_n['wma_20']     #W-Moving Average Cross Over

  #Exponential Moving Averages of Average Returns
  test_data_n['ema_4h']=ta.EMA(test_data_n['Mean_12'], 16)                #4 Hour Moving Average
  test_data_n['ema_8h']=ta.EMA(test_data_n['Mean_12'], 32)                #8 Hour Moving Average  
  test_data_n['ema_12h']=ta.EMA(test_data_n['Mean_12'], 48)               #12 Hour Moving Average
  test_data_n['ema_24h']=ta.EMA(test_data_n['Mean_12'], 96)               #1 Day Moving Average
  test_data_n['ema_12']=ta.EMA(test_data_n['Mean_12'], 1152)              #12 Day Exponential Moving Average
  test_data_n['ema_26']=ta.EMA(test_data_n['Mean_12'], 2496)              #26 Day Exponential Moving Average

  #Standard Deviation of Returns 
  test_data_n['std_24h']=ta.STDDEV(test_data_n['Mean_12'], 96)            #1 Day Standard Deviation
  test_data_n['std_10']=ta.STDDEV(test_data_n['Mean_12'], 960)            #10 Day Standard Deviation
  test_data_n['std_10']=ta.STDDEV(test_data_n['Mean_12'], 1920)           #20 Day Standard Deviation

  #Normalized Average True Range 
  test_data_n['NATR']=ta.NATR(test_data_n['high'], test_data_n['low'],test_data_n['close'],288)         #3 Day Normalized Average True Range

  #Rate of Change 
  test_data_n['roc_20']=ta.ROC(test_data_n['close'], 20)                         #20 Period Closing Price Rate of Change

  #Percentage Price Oscillator
  test_data_n['ppo_12_26']=ta.PPO(test_data_n['close'], 12,26)                   #PPO using 12 and 26 period moving averages

  #Relative Strength Index
  test_data_n['rsi_10']=ta.RSI(test_data_n['close'],960)

  #Average Directional Index

  test_data_n['adx']=ta.ADX(test_data_n['high'], test_data_n['low'], test_data_n['close'], 672)              #7 Day ADX
  test_data_n['adx_14']=ta.ADX(test_data_n['high'], test_data_n['low'], test_data_n['close'], 1344)          #14 Day ADX

  #Aroon Indicator
  test_data_n['aroon_down'], test_data_n['aroon_up']=ta.AROON(test_data_n['high'], test_data_n['close'], 26)           #26 Period Aroon Indicator

  #Aroon Oscillator
  test_data_n['aroon_osc']=ta.AROONOSC(test_data_n['high'], test_data_n['low'], 26)                                      #26 Period Aroon Oscillator

  #Trade Signals-Aroon Oscillator (Upward and Downward Trend Prices)
  test_data_n['price_trend_aroon']=np.where((test_data_n['aroon_osc']>=-70) &(test_data_n['aroon_osc']<=70),
                                        2,np.where(test_data_n['aroon_osc']<-70,0,np.where(test_data_n['aroon_osc']>70,1,np.nan)))   #Strong Price Trend (1)/Weak Price Trend (0)/Absence of Price Trend (2)

  #Bolllinger Bands 
  test_data_n['upperBB'], test_data_n['middleBB'], test_data_n['lowerBB'] = ta.BBANDS(test_data_n['Mean_12'], 2, 2, 20) #20 Period Bollinger Bands of Average Returns with 2 Standard Deviations 

  #14-Day Fast-Stochastic Oscillator Indicator (%K) and 3-Day Moving Average of Fast-Stochastic Oscillator Indicator (%D)
  test_data_n['fastk'], test_data_n['fastd'] = ta.STOCHF(test_data_n['high'], test_data_n['low'], test_data_n['close'], fastk_period=5, fastd_period=3, fastd_matype=0)

  #Overbought and Oversold Currency Signal based on Fast-Stochastic Oscillator
  test_data_n['kfast_ob_os']=np.where(test_data_n['fastk']>80, 1, np.where(test_data_n['fastk']<20,0,2))  #Overbought (1)/Oversold (0)/Hold (2)

  #Normalized On-Balance Volume
  test_data_n['obv']=ta.OBV(test_data_n['close'], test_data_n['volume'])
  test_data_n['volume_cumulative']=np.cumsum(test_data_n['volume'])                      #Divide Current OBV with Cumulative Volume
  test_data_n['obv_norm']=(test_data_n['obv']/test_data_n['volume_cumulative'])*100      #Ranges between -100 and 100 

  test_data1_final=test_data1_final.append(test_data_n)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
#Summary Statistics of Features

#Numeric Variables
test_data1_final.describe([0.1,0.25,0.5,0.75,.99]).transpose().to_csv('summary_stats_test1.csv')

#Missing Values
((test_data1_final.isna().sum())/len(test_data1_final)).transpose().to_csv('missing_test1.csv')

#Categorical Variables
display(test_data1_final['price_trend_aroon'].value_counts()/len(test_data1_final))

display(test_data1_final['kfast_ob_os'].value_counts()/len(test_data1_final))

2.0    0.687551
1.0    0.157263
0.0    0.150217
Name: price_trend_aroon, dtype: float64

2    0.600025
1    0.207034
0    0.192941
Name: kfast_ob_os, dtype: float64

In [ ]:
test_data1_final.shape

(38175, 65)

**Missing Values Handling and Features Exclusion**

In [ ]:
#Drop Missing Observations in Data
test_data1_final=test_data1_final.dropna(axis=0)
len(test_data1_final)

#Drop Unwanted Features

test_data1_final=test_data1_final.drop(['low','high','close','date_time','volume','ATR','stop_loss_a_c','take_profit_a_c','stop_loss_a_r','take_profit_a_r'],axis=1)

38175

In [ ]:
#Export the Final Data set

test_data1_final.to_excel('test_data1_final.xlsx')

**Test Data 2 Feature Engineering**

In [ ]:
test_data2=pd.read_excel(r'test_data2.xlsx')

test_data2=test_data2.drop(['Unnamed: 0'],axis=1)

**Feature Engineering**

In [ ]:
#(Pooled Data-Estimators)

test_data2_final=pd.DataFrame()

for pair in test_data2['pair'].unique():
  test_data_n=test_data2[test_data2['pair']==pair]
#Moving Averages of Average Returns

  test_data_n['ma_4h']=ta.SMA(test_data_n['Mean_12'], 16)                #4 Hour Moving Average
  test_data_n['ma_8h']=ta.SMA(test_data_n['Mean_12'], 32)                #8 Hour Moving Average  
  test_data_n['ma_12h']=ta.SMA(test_data_n['Mean_12'], 48)               #12 Hour Moving Average
  test_data_n['ma_24h']=ta.SMA(test_data_n['Mean_12'], 96)               #1 Day Moving Average
  test_data_n['ma_9']=ta.SMA(test_data_n['Mean_12'], 864)                #9 Day Moving Average
  test_data_n['ma_20']=ta.SMA(test_data_n['Mean_12'], 1920)              #20 Day Moving Average 
  test_data_n['ma_cross']=test_data_n['ma_9']-test_data_n['ma_20']      #Moving Average Cross Over

  #Weighted Moving Averages of Average Returns

  test_data_n['wma_4h']=ta.WMA(test_data_n['Mean_12'], 16)                #4 Hour W-Moving Average
  test_data_n['wma_8h']=ta.WMA(test_data_n['Mean_12'], 32)                #8 Hour W-Moving Average  
  test_data_n['wma_12h']=ta.WMA(test_data_n['Mean_12'], 48)               #12 Hour W-Moving Average
  test_data_n['wma_24h']=ta.WMA(test_data_n['Mean_12'], 96)               #1 Day W-Moving Average
  test_data_n['wma_9']=ta.WMA(test_data_n['Mean_12'], 864)                #9 Day W-Moving Average
  test_data_n['wma_20']=ta.WMA(test_data_n['Mean_12'], 1920)              #20 Day W-Moving Average 
  test_data_n['wma_cross']=test_data_n['wma_9']-test_data_n['wma_20']     #W-Moving Average Cross Over

  #Exponential Moving Averages of Average Returns
  test_data_n['ema_4h']=ta.EMA(test_data_n['Mean_12'], 16)                #4 Hour Moving Average
  test_data_n['ema_8h']=ta.EMA(test_data_n['Mean_12'], 32)                #8 Hour Moving Average  
  test_data_n['ema_12h']=ta.EMA(test_data_n['Mean_12'], 48)               #12 Hour Moving Average
  test_data_n['ema_24h']=ta.EMA(test_data_n['Mean_12'], 96)               #1 Day Moving Average
  test_data_n['ema_12']=ta.EMA(test_data_n['Mean_12'], 1152)              #12 Day Exponential Moving Average
  test_data_n['ema_26']=ta.EMA(test_data_n['Mean_12'], 2496)              #26 Day Exponential Moving Average

  #Standard Deviation of Returns 
  test_data_n['std_24h']=ta.STDDEV(test_data_n['Mean_12'], 96)            #1 Day Standard Deviation
  test_data_n['std_10']=ta.STDDEV(test_data_n['Mean_12'], 960)            #10 Day Standard Deviation
  test_data_n['std_10']=ta.STDDEV(test_data_n['Mean_12'], 1920)           #20 Day Standard Deviation

  #Normalized Average True Range 
  test_data_n['NATR']=ta.NATR(test_data_n['high'], test_data_n['low'],test_data_n['close'],288)         #3 Day Normalized Average True Range

  #Rate of Change 
  test_data_n['roc_20']=ta.ROC(test_data_n['close'], 20)                         #20 Period Closing Price Rate of Change

  #Percentage Price Oscillator
  test_data_n['ppo_12_26']=ta.PPO(test_data_n['close'], 12,26)                   #PPO using 12 and 26 period moving averages

  #Relative Strength Index
  test_data_n['rsi_10']=ta.RSI(test_data_n['close'],960)

  #Average Directional Index

  test_data_n['adx']=ta.ADX(test_data_n['high'], test_data_n['low'], test_data_n['close'], 672)              #7 Day ADX
  test_data_n['adx_14']=ta.ADX(test_data_n['high'], test_data_n['low'], test_data_n['close'], 1344)          #14 Day ADX

  #Aroon Indicator
  test_data_n['aroon_down'], test_data_n['aroon_up']=ta.AROON(test_data_n['high'], test_data_n['close'], 26)           #26 Period Aroon Indicator

  #Aroon Oscillator
  test_data_n['aroon_osc']=ta.AROONOSC(test_data_n['high'], test_data_n['low'], 26)                                      #26 Period Aroon Oscillator

  #Trade Signals-Aroon Oscillator (Upward and Downward Trend Prices)
  test_data_n['price_trend_aroon']=np.where((test_data_n['aroon_osc']>=-70) &(test_data_n['aroon_osc']<=70),
                                        2,np.where(test_data_n['aroon_osc']<-70,0,np.where(test_data_n['aroon_osc']>70,1,np.nan)))   #Strong Price Trend (1)/Weak Price Trend (0)/Absence of Price Trend (2)

  #Bolllinger Bands 
  test_data_n['upperBB'], test_data_n['middleBB'], test_data_n['lowerBB'] = ta.BBANDS(test_data_n['Mean_12'], 2, 2, 20) #20 Period Bollinger Bands of Average Returns with 2 Standard Deviations 

  #14-Day Fast-Stochastic Oscillator Indicator (%K) and 3-Day Moving Average of Fast-Stochastic Oscillator Indicator (%D)
  test_data_n['fastk'], test_data_n['fastd'] = ta.STOCHF(test_data_n['high'], test_data_n['low'], test_data_n['close'], fastk_period=5, fastd_period=3, fastd_matype=0)

  #Overbought and Oversold Currency Signal based on Fast-Stochastic Oscillator
  test_data_n['kfast_ob_os']=np.where(test_data_n['fastk']>80, 1, np.where(test_data_n['fastk']<20,0,2))  #Overbought (1)/Oversold (0)/Hold (2)

  #Normalized On-Balance Volume
  test_data_n['obv']=ta.OBV(test_data_n['close'], test_data_n['volume'])
  test_data_n['volume_cumulative']=np.cumsum(test_data_n['volume'])                      #Divide Current OBV with Cumulative Volume
  test_data_n['obv_norm']=(test_data_n['obv']/test_data_n['volume_cumulative'])*100      #Ranges between -100 and 100 

  test_data2_final=test_data2_final.append(test_data_n)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
#Summary Statistics of Features

#Numeric Variables
test_data2_final.describe([0.1,0.25,0.5,0.75,.99]).transpose().to_csv('summary_stats_test2.csv')

#Missing Values
((test_data2_final.isna().sum())/len(test_data2_final)).transpose().to_csv('missing_test2.csv')

#Categorical Variables
display(test_data2_final['price_trend_aroon'].value_counts()/len(test_data2_final))

display(test_data2_final['kfast_ob_os'].value_counts()/len(test_data2_final))

2.0    0.702396
1.0    0.161478
0.0    0.130302
Name: price_trend_aroon, dtype: float64

2    0.612542
1    0.214752
0    0.172706
Name: kfast_ob_os, dtype: float64

**Missing Values Handling and Feature Exclusion**

In [ ]:
#Drop Missing Observations in Data
test_data2_final=test_data2_final.dropna(axis=0)
len(test_data2_final)

#Drop Unwanted Features

test_data2_final=test_data2_final.drop(['low','high','close','date_time','volume','ATR','stop_loss_a_c','take_profit_a_c','stop_loss_a_r','take_profit_a_r'],axis=1)

KeyError: ignored

In [ ]:
#Export the Final Data set

test_data2_final.to_excel('test_data2_final.xlsx')